In [1]:
# Install the necessary libraries
!pip install transformers datasets evaluate accelerate
!pip install torch
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.5 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [2]:
# Import the necessary libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset, Dataset
import csv
import evaluate

In [3]:
# Import the necessary dataset
data = load_dataset("wmt16", name="de-en", split="test")

README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/282M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/267M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/277M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/475k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [4]:
# Import a pretrained open-mt-en-de model and create a translator from the pipeline
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-de")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-de")
translator = pipeline("translation_en_to_de", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
# Translate the sentences (around 2K) in the dataset and write them into a file
file = open('open_mt_translation.csv', 'w')
writer = csv.writer(file)
writer.writerow(['Sample Input', 'Translation', 'Ground Truth'])
for i in range(0, len(data)):
    text = data[i]["translation"]["en"]
    output = translator(text)
    #print(output)
    #print(output[0]["translation_text"])
    prediction = output[0]["translation_text"]
    ground_truth = data[i]["translation"]["de"]
    row = [text, prediction, ground_truth]
    writer.writerow(row)
file.close()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [6]:
# Read the predictions and references from the created csv file
filename = "open_mt_translation.csv"
predictions = []
references = []
with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    i = 0
    for row in csvreader:
        if i == 0:
            i = i + 1
            continue
        else:
            predictions.append(row[1])
            references.append(row[2])

In [7]:
# Evaluate the bleu metric
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print("Bleu Metric:", results["bleu"])

Bleu Metric: 0.36849511475932595


In [8]:
# Evaluate the meteor metric
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=predictions, references=references)
print("Meteor Metric", results['meteor'])

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


Meteor Metric 0.630451864475214
